# Домашняя работа

1. Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
2. Построить случайный лес, вывести важность признаков
3. Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
4. В качестве решения: Jupyter notebook с кодом, комментариями и графиками

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
#Формируем датафрейм
df_test = pd.read_csv('data/test.csv') #тестовые данные
df_sample_submission = pd.read_csv('data/sample_submission.csv') #цены по df_test
df_trial = pd.read_csv('data/train.csv') #тренировочные даные
#Основной датафрейм
df = pd.concat([df_trial, df_test.merge( df_sample_submission )], axis = 0, ignore_index = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             2919 non-null   int64  
 1   MSSubClass     2919 non-null   int64  
 2   MSZoning       2915 non-null   object 
 3   LotFrontage    2433 non-null   float64
 4   LotArea        2919 non-null   int64  
 5   Street         2919 non-null   object 
 6   Alley          198 non-null    object 
 7   LotShape       2919 non-null   object 
 8   LandContour    2919 non-null   object 
 9   Utilities      2917 non-null   object 
 10  LotConfig      2919 non-null   object 
 11  LandSlope      2919 non-null   object 
 12  Neighborhood   2919 non-null   object 
 13  Condition1     2919 non-null   object 
 14  Condition2     2919 non-null   object 
 15  BldgType       2919 non-null   object 
 16  HouseStyle     2919 non-null   object 
 17  OverallQual    2919 non-null   int64  
 18  OverallC

In [3]:
#Класс для очистики датафрейм 
#Категориальные данные преобразуем по O H E (унимодальное кодирование)
#Пропуски заменяем округленным до целого средним

class cls2task_prepdre_df:
    
    ret_df = pd.DataFrame()
    
    def __init__(self, df_):
        self.df      = df_
        #list(df.dtypes[df.dtypes == object].index)
        self.df_num = df_.select_dtypes(exclude=['object']).copy()
        self.df_obj = df_.select_dtypes(include=['object']).copy()
    
    def ohe(self):
        ret = pd.DataFrame()
        for col in self.df_obj.columns:
            _ = pd.get_dummies(self.df_obj[col], prefix=col, drop_first=False)
            ret =  pd.concat([ret, _], axis=1)
        return ret
    
    def set_mean(self):
        for col in self.ret_df.columns:
            if self.ret_df[col].isna().sum() > 0:
                _ = self.ret_df[[col]].mean().round(0)
                self.ret_df.loc[ self.ret_df[ self.ret_df[col].isna()].index, col] = _[0]                
        return 0
    
    def main(self):
        self.ret_df = pd.concat([self.df_num, self.ohe()], axis=1)
        self.set_mean()
        return self.ret_df
    

# Случайный лес


**Важные гиперпараметры алгоритма**

a. Параметры деревьев
1. criterion - критерий построения дерева
2. max_depth - максимальная глубина дерева (обычно 10-20, больше глубина -> больше риск переобучения)
3. min_samples_leaf - минимальное число объектов в листе (обычно 20+, больше объектов -> меньше риск переобучения)

b. Параметры леса
1. n_estimators - кол-во деревьев (чем больше тем лучше)
2. max_features - число признаков случайного подпространства
3. n_jobs - кол-во потоков для одновременного построения деревьев (большая прибавка к скорости на многоядерных процах)

In [4]:
#Обрабатываем основной датафрейм
transform_df = cls2task_prepdre_df(df.iloc[:,1:-1]) #из обработки иключаем 1 (Id) и послденее поле (SalePrice)
transform_df.main()

#Признаковое пространство
X = transform_df.ret_df
#Target-значение (цена)
y = df.iloc[:,-1].round(0)
#Создаем и обучаем модель
model_rf = RandomForestClassifier(n_estimators=100, max_depth=25, min_samples_leaf=25, max_features=0.5, n_jobs=-1)
model_rf.fit(X, y)

RandomForestClassifier(max_depth=25, max_features=0.5, min_samples_leaf=25,
                       n_jobs=-1)

In [5]:
#Важность признака (%)
ia_df = pd.DataFrame( {'Attribute':X.columns, \
                       'Importances':100*model_rf.feature_importances_} ).\
           sort_values(by = 'Importances', ascending = False).reset_index()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also  
    print( ia_df )


     index              Attribute  Importances
0       15              GrLivArea     7.559181
1        2                LotArea     6.059605
2       26             GarageArea     5.929530
3       11            TotalBsmtSF     5.684786
4       12               1stFlrSF     5.354006
5       10              BsmtUnfSF     5.039145
6        8             BsmtFinSF1     4.229846
7        1            LotFrontage     3.823546
8        5              YearBuilt     3.727047
9        6           YearRemodAdd     2.905590
10      28            OpenPorchSF     2.745460
11      27             WoodDeckSF     2.740743
12      13               2ndFlrSF     2.700098
13      24            GarageYrBlt     2.644173
14      34                 MoSold     2.437204
15       7             MasVnrArea     1.959401
16       3            OverallQual     1.712475
17      35                 YrSold     1.412917
18      22           TotRmsAbvGrd     1.253023
19      18               FullBath     1.174700
20       9   

# Стекинг 3-х моделей

In [6]:
# Библиотеки машинного обучения
from sklearn.ensemble import StackingClassifier #Стекинг
from sklearn.linear_model import LogisticRegression #Логистическая регрессия
from sklearn.neighbors import KNeighborsClassifier #К-ближайших соседей
from sklearn.tree import DecisionTreeClassifier #Дерево решений

In [ ]:
#Ансамбль моделей
classifier = StackingClassifier(
    [
        ('lr', LogisticRegression()),
        ('dt', DecisionTreeClassifier()),
        ('kn', KNeighborsClassifier()),
    ],
KNeighborsClassifier() )
#Делим выборку на тренировочную и тестовую (в отбор включаем все значимые призанки по условию значимости  > 0.00001%)
X_train, X_test, y_train, y_test = train_test_split(X[ia_df[ia_df['Importances'] > 0.00001 ]['Attribute']], y, test_size=0.3)

#Нормирование данных
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
#Обучение ансабля
classifier.fit(X_train, y_train)

D:\Anakonda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\Anakonda\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
D:\Anakonda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.

In [ ]:
scaler.fit(X_test)
X_test = scaler.transform(X_test)
y_pred = classifier.predict(X_test)
print('Прогноз цен на недвижимость (сравнение c фактом):')
list( zip(y_test, y_pred) )